In [3]:
import pickle
from qiskit_ibm_provider import IBMProvider
from qiskit.quantum_info import hellinger_fidelity
from evaluation_metrics import TV
import numpy as np
from math import pow
import yaml
import logging.config
import logging
import pandas as pd
from openpyxl.formatting.rule import ColorScaleRule
from math import pow
from openpyxl.styles import Alignment
from openpyxl.styles import Border, Side
from qiskit_aer import AerSimulator

In [4]:
#Setup logger
with open('logging.yaml', encoding="utf8") as ly:
    loggingDict = yaml.safe_load(ly)
logging.config.dictConfig(loggingDict)
logger = logging.getLogger("driver_code_ibm")

In [5]:
with open('../circuits/job_params_granular_T.pkl', 'rb') as f:
    job_params = pickle.load(f)

with open('../circuits/job_params_granular_T_sampler.pkl', 'rb') as f:
    job_params_sampler = pickle.load(f)

In [6]:
systems = {i['system'] for i in job_params}
print(systems)

{'ibm_sherbrooke', 'ibm_brisbane', 'ibm_kyoto', 'ibm_nazca'}


In [7]:
{i['system'] for i in job_params_sampler}

{'ibm_brisbane', 'ibm_kyoto', 'ibm_nazca', 'ibm_sherbrooke'}

In [8]:
with open('../results/ideal_results.pkl', 'rb') as f:
    ideal_results = pickle.load(f)

with open('../../ibm_API_key','r') as file:
    token = file.readline()

provider = IBMProvider(token=token, instance="ibm-q-ncsu/nc-state/quantum-compiler")

In [7]:
ideal_results.keys()

dict_keys([(4, 0.1), (4, 0.2), (4, 0.3), (4, 0.4), (4, 0.5), (4, 0.6), (4, 0.7), (4, 0.8), (4, 0.9), (4, 1.0), (4, 1.5), (4, 2.0), (6, 0.1), (6, 0.2), (6, 0.3), (6, 0.4), (6, 0.5), (6, 0.6), (6, 0.7), (6, 0.8), (6, 0.9), (6, 1.0), (6, 1.5), (6, 2.0), (8, 0.1), (8, 0.2), (8, 0.3), (8, 0.4), (8, 0.5), (8, 0.6), (8, 0.7), (8, 0.8), (8, 0.9), (8, 1.0), (8, 1.5), (8, 2.0), (10, 0.1), (10, 0.2), (10, 0.3), (10, 0.4), (10, 0.5), (10, 0.6), (10, 0.7), (10, 0.8), (10, 0.9), (10, 1.0), (10, 1.5), (10, 2.0)])

In [27]:
#Process each of the noise level results
def process_results(result,layout):

    def layout_rev(res):
        n = len(layout)
        # print(self.layout)
        b = res
        ret = ""
        for i in range(n):
            ret += b[-1 - layout[i]]
        return ret


    n_shots = sum(result.values())

    from collections import defaultdict

    def default_value():
        return 0

    ret = defaultdict(default_value)

    for key in result.keys():
        new_key = layout_rev(key)
        ret[new_key] += result[key] / n_shots

    return ret

#ZNE function
def zne(dists,degree, number_of_points, top=65,percentage_mode=False):

    if percentage_mode:
        number_of_keys = (len(dists[0].keys()) * top) // 100
    else:
        number_of_keys = top

    #Getting the top 10 results from the distribution
    sorted_dist = dict(sorted(dists[0].items(), key=lambda x: x[1], reverse=True))
    top_k = list(sorted_dist.keys())[:number_of_keys]
    remaining_keys = list(sorted_dist.keys())[number_of_keys:]

    #Since this was global folding
    x_axis = np.array([i for i in range(1,number_of_points * 2,2)])

    #Extrapolation of top 10 results
    mitigated_top_k_dist = {}

    for key in top_k:
        mitigated_top_k_dist[key] = 0
        y_axis = np.array([ dists[i][key] if key in dists[i] else 0 for i in range(0,number_of_points)])
        coefficients = np.polyfit(x_axis, y_axis, degree)
        poly = np.poly1d(coefficients)
        mitigated_top_k_dist[key] = 0 if poly(0) < 0 else poly(0)

    #Calculate the sum of the mitigated top 10 distribution
    sum_mitigated_top_k_dist = sum(mitigated_top_k_dist.values())
    from functools import reduce
    sum_remaining_keys = reduce(lambda acc, key_value: acc + key_value[1] if key_value[0] in remaining_keys else acc, sorted_dist.items(), 0)
    total_sum = sum_mitigated_top_k_dist + sum_remaining_keys

    #Normalize the mitigated top 10 distribution
    normalized_mitigated_top_k_dist = dict(map(lambda x: (x[0],x[1]/total_sum),mitigated_top_k_dist.items()))
    normalized_mitigated_top_k_dist.update(dict(map(lambda x: (x[0],x[1]/total_sum),list(filter(lambda x: x[0] in remaining_keys,sorted_dist.items())))))
    return normalized_mitigated_top_k_dist

#Function for performing folding free ZNE
def folding_free_zne(dist,reliability,N,top=65,percentage_mode=False):

    if percentage_mode:
        number_of_keys = (len(dist.keys()) * top) // 100
    else:
        number_of_keys = top

    #Getting the top 10 results from the distribution
    sorted_dist = dict(sorted(dist.items(), key=lambda x: x[1], reverse=True))
    top_k = list(sorted_dist.keys())[:number_of_keys]
    remaining_keys = list(sorted_dist.keys())[number_of_keys:]

    x_axis = [(1-reliability),1]

    #Extrapolation of top k results
    mitigated_top_k_dist = {}

    for key in top_k:
        mitigated_top_k_dist[key] = 0
        y_axis = [dist[key] if key in dist else 0,pow(2,-N)]
        coefficients = np.polyfit(x_axis, y_axis, 2)
        poly = np.poly1d(coefficients)
        mitigated_top_k_dist[key] = 0 if poly(0) < 0 else poly(0)

    #Calculate the sum of the mitigated top 10 distribution
    sum_mitigated_top_k_dist = sum(mitigated_top_k_dist.values())
    from functools import reduce
    sum_remaining_keys = reduce(lambda acc, key_value: acc + key_value[1] if key_value[0] in remaining_keys else acc, sorted_dist.items(), 0)
    total_sum = sum_mitigated_top_k_dist + sum_remaining_keys

    #Normalize the mitigated top 10 distribution
    normalized_mitigated_top_k_dist = dict(map(lambda x: (x[0],x[1]/total_sum),mitigated_top_k_dist.items()))
    normalized_mitigated_top_k_dist.update(dict(map(lambda x: (x[0],x[1]/total_sum),list(filter(lambda x: x[0] in remaining_keys,sorted_dist.items())))))

    return normalized_mitigated_top_k_dist

#Function for performing reliability based ZNE
def reliability_based_zne(dists,N,reliabilities,number_of_points,top=65,percentage_mode=False):

    if percentage_mode:
        number_of_keys = (len(dists[0].keys()) * top) // 100
    else:
        number_of_keys = top

    #Getting the top 10 results from the distribution
    sorted_dist = dict(sorted(dists[0].items(), key=lambda x: x[1], reverse=True))
    top_k = list(sorted_dist.keys())[:number_of_keys]
    remaining_keys = list(sorted_dist.keys())[number_of_keys:]

    x_axis = list(map(lambda x: 1-x, reliabilities))
    x_axis = x_axis[:number_of_points] + [1]
    #x_axis = x_axis[:number_of_points]

    #Extrapolation of top 10 results
    mitigated_top_k_dist = {}

    for key in top_k:
        mitigated_top_k_dist[key] = 0
        y_axis = np.array([dists[i][key] if key in dists[i] else 0 for i in range(0,number_of_points)] + [pow(2,-N)]) 
        #y_axis = np.array([dists[i][key] if key in dists[i] else 0 for i in range(0,number_of_points)])
        coefficients = np.polyfit(x_axis, y_axis, 2)
        poly = np.poly1d(coefficients)
        mitigated_top_k_dist[key] = 0 if poly(0) < 0 else poly(0)

    #Calculate the sum of the mitigated top 10 distribution
    sum_mitigated_top_k_dist = sum(mitigated_top_k_dist.values())
    from functools import reduce
    sum_remaining_keys = reduce(lambda acc, key_value: acc + key_value[1] if key_value[0] in remaining_keys else acc, sorted_dist.items(), 0)
    total_sum = sum_mitigated_top_k_dist + sum_remaining_keys

    #Normalize the mitigated top 10 distribution
    normalized_mitigated_top_10_dist = dict(map(lambda x: (x[0],x[1]/total_sum),mitigated_top_k_dist.items()))
    normalized_mitigated_top_10_dist.update(dict(map(lambda x: (x[0],x[1]/total_sum),list(filter(lambda x: x[0] in remaining_keys,sorted_dist.items())))))

    return normalized_mitigated_top_10_dist

def simulate(circ, backend, with_noise):
    if with_noise:
        # load noise model
        with open(f'noise_models/{backend.name}_noise_model.pkl', 'rb') as f:
            noise_model = pickle.load(f)

        # create a simulator for backend with noise model
        basis_gates = noise_model.basis_gates
        coupling_map = backend.configuration().coupling_map

        sim = AerSimulator(noise_model=noise_model,
                        basis_gates=basis_gates,
                        coupling_map=coupling_map)
    else:
        # create a simulator for backend
        sim = AerSimulator()

    # simulate and extract results
    shots = 4096
    simulator_result = sim.run(circ, shots = shots, seed_simulator=12345).result()
    simulator_counts = simulator_result.get_counts()

    #Reverse the key strings
    simulator_counts = {k[::-1]: v/shots for k, v in simulator_counts.items()}

    return simulator_counts

### Initial

In [5]:
results = []

for job_param in job_params:
    
    logger.info("Processing for N = {}, T = {}, system = {}".format(job_param['N'],job_param['T'],job_param['system']))
    result = {'N': job_param['N'], 'T': job_param['T'], 'system': job_param['system']}

    #Getting the layout
    circuit_filename = "../circuits/" + job_param['system'] + "_" + str(job_param['N']) + "_" + str(job_param['T']) + ".pkl"
    with open(circuit_filename, 'rb') as f:
        circuit = pickle.load(f)
    layout = circuit['layout']

    logger.info("Layout: {}".format(layout))
    result['layout'] = layout

    #Get the raw results
    raw_results = provider.retrieve_job(job_param['job_id']).result().get_counts()
    processed_results = list(map(lambda x: process_results(x,layout),raw_results))
    ideal_result = ideal_results[job_param['N'],job_param['T']]

    hellinger_fidelities = list(map(lambda x: hellinger_fidelity(ideal_result,x),processed_results))
    TVs = list(map(lambda x: TV(ideal_result,x),processed_results))

    logger.info("")
    logger.info("Initial Results:")
    logger.info("Hellinger Fidelities: {} Max: {}".format(hellinger_fidelities, max(hellinger_fidelities)))
    logger.info("TVs: {} Min: {}".format(TVs, min(TVs)))
    logger.info("")

    result['initial'] = {}
    result['initial']['hellinger_fidelities'] = hellinger_fidelities
    result['initial']['max_hellinger_fidelity'] = max(hellinger_fidelities)
    result['initial']['TVs'] = TVs
    result['initial']['min_TV'] = min(TVs)

    #Get the ZNE results
    logger.info("Getting ZNE results")
    result['zne'] = {'hellinger_fidelity': {}, 'TV': {}}

    max_hellinger_fidelity = 0
    max_degree = 0
    max_number_of_points = 0

    min_tv = 1
    min_degree = 0
    min_number_of_points = 0

    for degree in range(1,2):
        for number_of_points in range(2,6):
            zne_processed_results = zne(processed_results,degree,number_of_points)
            result['zne']['hellinger_fidelity'][(degree,number_of_points)] = hellinger_fidelity(zne_processed_results,ideal_result)
            result['zne']['TV'][(degree,number_of_points)] = TV(zne_processed_results,ideal_result)

            if result['zne']['hellinger_fidelity'][(degree,number_of_points)] > max_hellinger_fidelity:
                max_hellinger_fidelity = result['zne']['hellinger_fidelity'][(degree,number_of_points)]
                max_degree = degree
                max_number_of_points = number_of_points

            if result['zne']['TV'][(degree,number_of_points)] < min_tv:
                min_tv = result['zne']['TV'][(degree,number_of_points)]
                min_degree = degree
                min_number_of_points = number_of_points

            logger.info("Degree: {}, Number of Points: {}, Hellinger Fidelity: {}, TV: {}".format(degree,number_of_points,result['zne']['hellinger_fidelity'][(degree,number_of_points)],result['zne']['TV'][(degree,number_of_points)]))
    
    logger.info("")
    logger.info("ZNE completed:")
    result['zne']['hellinger_fidelity']['max_hellinger_fidelity'] = max_hellinger_fidelity
    result['zne']['hellinger_fidelity']['max_degree'] = max_degree
    result['zne']['hellinger_fidelity']['max_number_of_points'] = max_number_of_points
    result['zne']['TV']['min_tv'] = min_tv
    result['zne']['TV']['min_degree'] = min_degree
    result['zne']['TV']['min_number_of_points'] = min_number_of_points
    logger.info("Max Hellinger Fidelity: {}, Degree: {}, Number of Points: {}".format(max_hellinger_fidelity,max_degree,max_number_of_points))
    logger.info("Min TV: {}, Degree: {}, Number of Points: {}".format(min_tv,min_degree,min_number_of_points))
    logger.info("")

    #Get the folding free ZNE results
    logger.info("Getting Folding Free ZNE results")
    result['ffzne'] = {}
    ffzne_mitigated_results = folding_free_zne(processed_results[0],job_param['N'],job_param['T'],job_param['system'])
    result['ffzne']['hellinger_fidelity'] = hellinger_fidelity(ffzne_mitigated_results,ideal_result)
    result['ffzne']['TV'] = TV(ffzne_mitigated_results,ideal_result)

    logger.info("")
    logger.info("FFZNE completed:")
    logger.info("Hellinger Fidelity: {}, TV: {}".format(result['ffzne']['hellinger_fidelity'],result['ffzne']['TV']))
    logger.info("")

    #Get the reliability based ZNE results
    logger.info("Getting Reliability Based ZNE results")
    result['rbzne'] = {'hellinger_fidelity': {}, 'TV': {}}

    max_hellinger_fidelity = 0
    max_number_of_points = 0
    min_tv = 1
    min_number_of_points = 0
    
    for number_of_points in range(2,6):
        rbzne_mitigated_results = reliability_based_zne(processed_results,job_param['N'],job_param['T'],job_param['system'],number_of_points)
        result['rbzne']['hellinger_fidelity'][number_of_points] = hellinger_fidelity(rbzne_mitigated_results,ideal_result)
        result['rbzne']['TV'][number_of_points] = TV(rbzne_mitigated_results,ideal_result)

        if result['rbzne']['hellinger_fidelity'][number_of_points] > max_hellinger_fidelity:
            max_hellinger_fidelity = result['rbzne']['hellinger_fidelity'][number_of_points]
            max_number_of_points = number_of_points

        if result['rbzne']['TV'][number_of_points] < min_tv:
            min_tv = result['rbzne']['TV'][number_of_points]
            min_number_of_points = number_of_points

        logger.info("Number of Points: {}, Hellinger Fidelity: {}, TV: {}".format(number_of_points,result['rbzne']['hellinger_fidelity'][number_of_points],result['rbzne']['TV'][number_of_points]))
    
    logger.info("")
    logger.info("RBZNE completed:")
    result['rbzne']['hellinger_fidelity']['max_hellinger_fidelity'] = max_hellinger_fidelity
    result['rbzne']['hellinger_fidelity']['max_number_of_points'] = max_number_of_points
    result['rbzne']['TV']['min_tv'] = min_tv
    result['rbzne']['TV']['min_number_of_points'] = min_number_of_points
    logger.info("Max Hellinger Fidelity: {}, Number of Points: {}".format(max_hellinger_fidelity,max_number_of_points))
    logger.info("Min TV: {}, Number of Points: {}".format(min_tv,min_number_of_points))
    logger.info("=====================================================================================================")
    logger.info("")
    results.append(result)

In [14]:
updated_result = list(map(lambda x: {"N" : x['N'], "T" : x['T'], "system" : x['system'], 
                            "initial_helliger_fidelity" : x['initial']['max_hellinger_fidelity'],
                            "initial_TV" : x['initial']['min_TV'],
                            "zne_hellinger_fidelity" : x['zne']['hellinger_fidelity'][(1,3)],
                            #"zne_hel_degree" : x['zne']['hellinger_fidelity']['max_degree'] ,
                            #"zne_hel_number_of_points" : x['zne']['hellinger_fidelity']['max_number_of_points'],
                            "zne_TV" : x['zne']['TV'][(1,3)], 
                            #"zne_TV_degree" : x['zne']['TV']['min_degree'],
                            #"zne_TV_number_of_points" : x['zne']['TV']['min_number_of_points'],
                            "ffzne_hellinger_fidelity" : x['ffzne']['hellinger_fidelity'],
                            "ffzne_TV" : x['ffzne']['TV'], 
                            "rbzne_hellinger_fidelity" : x['rbzne']['hellinger_fidelity'][3], 
                            #"rbzne_hel_number_of_points" : x['rbzne']['hellinger_fidelity']['max_number_of_points'],
                            "rbzne_TV" : x['rbzne']['TV'][3],
                            #"rbzne_TV_number_of_points" : x['rbzne']['TV']['min_number_of_points']
                            },results))

In [15]:
import pandas as pd

df = pd.DataFrame(updated_result)
df['zne_hellinger_percentage_change'] = (df['zne_hellinger_fidelity'] - df['initial_helliger_fidelity'])/df['initial_helliger_fidelity']* 100
df['zne_TV_percentage_change'] = (df['initial_TV'] - df['zne_TV'])/df['initial_TV'] * 100
df['ffzne_hellinger_percentage_change'] = (df['ffzne_hellinger_fidelity'] - df['initial_helliger_fidelity'])/df['initial_helliger_fidelity']* 100
df['ffzne_TV_percentage_change'] = (df['initial_TV'] - df['ffzne_TV'])/df['initial_TV'] * 100
df['rbzne_hellinger_percentage_change'] = (df['rbzne_hellinger_fidelity'] - df['initial_helliger_fidelity'])/df['initial_helliger_fidelity']* 100
df['rbzne_TV_percentage_change'] = (df['initial_TV'] - df['rbzne_TV'])/df['initial_TV']* 100

#Arrange the columns properly
df = df[['N','T','system',
         'initial_helliger_fidelity',
         'initial_TV',
         'zne_hellinger_fidelity',
         'zne_hellinger_percentage_change',
         'zne_TV',
         'zne_TV_percentage_change',
         'ffzne_hellinger_fidelity',
         'ffzne_hellinger_percentage_change',
         'ffzne_TV',
         'ffzne_TV_percentage_change',
         'rbzne_hellinger_fidelity',
         'rbzne_hellinger_percentage_change',
         'rbzne_TV',
         'rbzne_TV_percentage_change']]

#Create dfs based on the system and post in different sheets of the excel
brisbane_df = df[df['system'] == 'ibm_brisbane']
kyoto_df = df[df['system'] == 'ibm_kyoto']

with pd.ExcelWriter("../results/ibm_results.xlsx") as writer:
    brisbane_df.to_excel(writer, sheet_name='Brisbane',index=False)
    kyoto_df.to_excel(writer, sheet_name='Kyoto',index=False)

### Varying the percentage of top k in zne and rbzne

In [19]:
logger.info("Starting the second set of results")

results = []

for job_param in job_params:
    
    logger.info("Processing for N = {}, T = {}, system = {}".format(job_param['N'],job_param['T'],job_param['system']))
    result = {'N': job_param['N'], 'T': job_param['T'], 'system': job_param['system']}

    #Getting the layout
    circuit_filename = "../circuits/" + job_param['system'] + "_" + str(job_param['N']) + "_" + str(job_param['T']) + ".pkl"
    with open(circuit_filename, 'rb') as f:
        circuit = pickle.load(f)
    layout = circuit['layout']

    logger.info("Layout: {}".format(layout))
    result['layout'] = layout

    #Get the raw results
    raw_results = provider.retrieve_job(job_param['job_id']).result().get_counts()
    processed_results = list(map(lambda x: process_results(x,layout),raw_results))
    ideal_result = ideal_results[job_param['N'],job_param['T']]

    hellinger_fidelities = list(map(lambda x: hellinger_fidelity(ideal_result,x),processed_results))
    TVs = list(map(lambda x: TV(ideal_result,x),processed_results))

    logger.info("")
    logger.info("Initial Results:")
    logger.info("Hellinger Fidelities: {} Max: {}".format(hellinger_fidelities, max(hellinger_fidelities)))
    logger.info("TVs: {} Min: {}".format(TVs, min(TVs)))
    logger.info("")

    result['initial'] = {}
    result['initial']['hellinger_fidelities'] = hellinger_fidelities
    result['initial']['max_hellinger_fidelity'] = max(hellinger_fidelities)
    result['initial']['TVs'] = TVs
    result['initial']['min_TV'] = min(TVs)

    #Get the ZNE results
    logger.info("Getting ZNE results")
    result['zne'] = {'hellinger_fidelity': {}, 'TV': {}}

    max_hellinger_fidelity = 0
    max_top_k = 0

    min_tv = 1
    min_top_k = 0

    for top_k in range(10,110,10):
        zne_processed_results = zne(processed_results,1,3,top_k)
        result['zne']['hellinger_fidelity'][top_k] = hellinger_fidelity(zne_processed_results,ideal_result)
        result['zne']['TV'][top_k] = TV(zne_processed_results,ideal_result)

        if result['zne']['hellinger_fidelity'][top_k] > max_hellinger_fidelity:
            max_hellinger_fidelity = result['zne']['hellinger_fidelity'][top_k]
            max_top_k = top_k

        if result['zne']['TV'][top_k] < min_tv:
            min_tv = result['zne']['TV'][top_k]
            min_top_k = top_k

        logger.info("Top K: {}, Hellinger Fidelity: {}, TV: {}".format(top_k,result['zne']['hellinger_fidelity'][top_k],result['zne']['TV'][top_k]))

    logger.info("")
    logger.info("ZNE completed:")
    result['zne']['hellinger_fidelity']['max_hellinger_fidelity'] = max_hellinger_fidelity
    result['zne']['hellinger_fidelity']['max_top_k'] = max_top_k
    result['zne']['TV']['min_tv'] = min_tv
    result['zne']['TV']['min_top_k'] = min_top_k
    logger.info("Max Hellinger Fidelity: {}, Top K: {}".format(max_hellinger_fidelity,max_top_k))
    logger.info("Min TV: {}, Top K: {}".format(min_tv,min_top_k))
    logger.info("")

    #Get the reliability based ZNE results
    logger.info("Getting Reliability Based ZNE results")
    result['rbzne'] = {'hellinger_fidelity': {}, 'TV': {}}

    max_hellinger_fidelity = 0
    max_top_k = 0
    min_tv = 1
    min_top_k = 0
    
    for top_k in range(10,110,10):
        rbzne_mitigated_results = reliability_based_zne(processed_results,job_param['N'],job_param['T'],job_param['system'],3,top_k)
        result['rbzne']['hellinger_fidelity'][top_k] = hellinger_fidelity(rbzne_mitigated_results,ideal_result)
        result['rbzne']['TV'][top_k] = TV(rbzne_mitigated_results,ideal_result)

        if result['rbzne']['hellinger_fidelity'][top_k] > max_hellinger_fidelity:
            max_hellinger_fidelity = result['rbzne']['hellinger_fidelity'][top_k]
            max_top_k = top_k

        if result['rbzne']['TV'][top_k] < min_tv:
            min_tv = result['rbzne']['TV'][top_k]
            min_top_k = top_k

        logger.info("Top K: {}, Hellinger Fidelity: {}, TV: {}".format(top_k,result['rbzne']['hellinger_fidelity'][top_k],result['rbzne']['TV'][top_k]))

    logger.info("")
    logger.info("RBZNE completed:")
    result['rbzne']['hellinger_fidelity']['max_hellinger_fidelity'] = max_hellinger_fidelity
    result['rbzne']['hellinger_fidelity']['max_top_k'] = max_top_k
    result['rbzne']['TV']['min_tv'] = min_tv
    result['rbzne']['TV']['min_top_k'] = min_top_k
    logger.info("Max Hellinger Fidelity: {}, Top K: {}".format(max_hellinger_fidelity,max_top_k))
    logger.info("Min TV: {}, Top K: {}".format(min_tv,min_top_k))
    logger.info("=====================================================================================================")
    logger.info("")
    results.append(result)

In [28]:
updated_result = list(map(lambda x: {"N_T_system" : (x['N'],x['T'],x['system']), 
                            "initial_helliger_fidelity" : x['initial']['max_hellinger_fidelity'],
                            "initial_TV" : x['initial']['min_TV'],
                            "zne_hellinger_fidelity" : x['zne']['hellinger_fidelity'],
                            "zne_TV" : x['zne']['TV'], 
                            "rbzne_hellinger_fidelity" : x['rbzne']['hellinger_fidelity'], 
                            "rbzne_TV" : x['rbzne']['TV']
                            },results))

In [61]:
top_k_hellinger = pd.DataFrame()

counter = 0

for N in range(4,11):
    for T in range(1,4):
        for system in ['ibm_brisbane','ibm_kyoto']:
            top_k_hellinger['{}_{}_{}'.format(N,T,system)] = updated_result[counter]['zne_hellinger_fidelity']
            counter += 1

top_k_tv = pd.DataFrame()

counter = 0

for N in range(4,11):
    for T in range(1,4):
        for system in ['ibm_brisbane','ibm_kyoto']:
            top_k_tv['{}_{}_{}'.format(N,T,system)] = updated_result[counter]['zne_TV']
            counter += 1

#Create dfs based on the system and post in different sheets of the excel
with pd.ExcelWriter("../results/top_k_comparisons.xlsx") as writer:
    top_k_hellinger.to_excel(writer, sheet_name='Hellinger Fidelity',index=False)
    top_k_tv.to_excel(writer, sheet_name='TV',index=False)

In [ ]:
reliability = []

for N in range(4,11):
    for T in range(1,4):
        for system in ['ibm_brisbane','ibm_kyoto','ibm_sherbrooke']:
            circuit_filename = "../circuits/modified/" + system + "_" + str(N) + "_" + str(T) + "_reliabilities.pkl"
            with open(circuit_filename, 'rb') as f:
                circuit = pickle.load(f)
                print(circuit)

            reliability.append({"N_T_system" : (N,T,system), "reliability" : circuit['circuit_reliabilities']})

In [ ]:
reliability_df = pd.DataFrame(reliability)
reliability

In [ ]:
reliability_df

In [78]:
#Write this to an excel file
reliability_df.to_excel("../results/reliabilities.xlsx",index=False)

### Considering case by case

#### Analyzing ibm_brisbane N=4 and T=1 case

In [ ]:
import pandas as pd
from openpyxl.formatting.rule import ColorScaleRule
from math import pow
from openpyxl.styles import Alignment
from openpyxl.styles import Border, Side

kyoto_job_params = list(filter(lambda x: x['system'] == 'ibm_kyoto',job_params))

summary_dict_list = []

for job_param in kyoto_job_params:

    circuit_filename = "../circuits/" + job_param['system'] + "_" + str(job_param['N']) + "_" + str(job_param['T']) + ".pkl"
    with open(circuit_filename, 'rb') as f:
        circuit = pickle.load(f)
    layout = circuit['layout']

    #Getting the ideal results
    ideal_result = ideal_results[job_param['N'],job_param['T']]

    #Getting the raw results
    raw_results = provider.retrieve_job(job_param['job_id']).result().get_counts()
    processed_results = list(map(lambda x: process_results(x,layout),raw_results))

    keys = set(ideal_result.keys()) | set(processed_results[0].keys()) | set(processed_results[1].keys()) | set(processed_results[2].keys()) | set(processed_results[3].keys()) | set(processed_results[4].keys())
    keys = sorted(list(keys))

    #Create a df and assign keys as index and name the column as State and add the values of keys to that column
    distribution_data_df = pd.DataFrame(keys,columns=['State'],index=keys)

    #Map the results to the dataframe
    distribution_data_df['Ideal probability'] = distribution_data_df['State'].map(ideal_result)
    distribution_data_df['Simulation probability'] = distribution_data_df['State'].map(processed_results[0])
    distribution_data_df['Noise factor 3'] = distribution_data_df['State'].map(processed_results[1])
    distribution_data_df['Noise factor 5'] = distribution_data_df['State'].map(processed_results[2])
    distribution_data_df['Noise factor 7'] = distribution_data_df['State'].map(processed_results[3])
    distribution_data_df['Noise factor 9'] = distribution_data_df['State'].map(processed_results[4])


    hellinger_fidelities = list(map(lambda x: hellinger_fidelity(ideal_result,x),processed_results))
    initial_hellinger_fidelity = hellinger_fidelities[0]

    TVs = list(map(lambda x: TV(ideal_result,x),processed_results))
    initial_TV = TVs[0]

    #ZNE
    max_hellinger_fidelity = 0
    max_number_of_points = 0
    max_degree = 0
    max_top = 0

    min_tv = 1
    min_number_of_points = 0
    min_degree = 0
    min_top = 0

    for top in [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100]:
        for degree in [1,2]:
            for number_of_points in [2,3,4,5]:
                zne_processed_results = zne(processed_results,degree,number_of_points,top)
                print("Top: {}, Degree: {}, Number of Points: {}, Hellinger Fidelity: {}, TV: {}".format(top,degree,number_of_points,hellinger_fidelity(zne_processed_results,ideal_result),TV(zne_processed_results,ideal_result)))

                if hellinger_fidelity(zne_processed_results,ideal_result) > max_hellinger_fidelity:
                    max_hellinger_fidelity = hellinger_fidelity(zne_processed_results,ideal_result)
                    max_number_of_points, max_degree, max_top = number_of_points, degree, top

                if TV(zne_processed_results,ideal_result) < min_tv:
                    min_tv = TV(zne_processed_results,ideal_result)
                    min_number_of_points, min_degree, min_top = number_of_points, degree, top

    zne_degree = max_degree
    zne_number_of_points = max_number_of_points
    zne_top = max_top

    print("ZNE completed")
    print("Max Hellinger Fidelity: {}, Number of Points: {}, Degree: {}, Top: {}".format(max_hellinger_fidelity,max_number_of_points,max_degree,max_top))
    print("Min TV: {}, Number of Points: {}, Degree: {}, Top: {}".format(min_tv,min_number_of_points,min_degree,min_top))

    zne_processed_results = zne(processed_results,zne_degree,zne_number_of_points,zne_top)

    best_zne_hellinger_fidelity = hellinger_fidelity(zne_processed_results,ideal_result)
    best_zne_tv = TV(zne_processed_results,ideal_result)

    print("Hellinger fidelity: {} TV: {}".format(hellinger_fidelity(zne_processed_results,ideal_result),TV(zne_processed_results,ideal_result)))

    distribution_data_df['ZNE best distribution'] = distribution_data_df['State'].map(zne_processed_results)

    #ffzne
    max_hellinger_fidelity = 0
    max_top = 0

    min_tv = 1
    min_top = 0

    for top in [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100]:
        ffzne_mitigated_results = folding_free_zne(processed_results[0],job_param['N'],job_param['T'],job_param['system'],top)
        print("Top: {}, Hellinger Fidelity: {}, TV: {}".format(top,hellinger_fidelity(ffzne_mitigated_results,ideal_result),TV(ffzne_mitigated_results,ideal_result)))

        if hellinger_fidelity(ffzne_mitigated_results,ideal_result) > max_hellinger_fidelity:
            max_hellinger_fidelity = hellinger_fidelity(ffzne_mitigated_results,ideal_result)
            max_top = top

        if TV(ffzne_mitigated_results,ideal_result) < min_tv:
            min_tv = TV(ffzne_mitigated_results,ideal_result)
            min_top = top

    ffzne_top = max_top

    print("FFZNE completed")
    print("Max Hellinger Fidelity: {}, Top: {}".format(max_hellinger_fidelity,max_top))
    print("Min TV: {}, Top: {}".format(min_tv,min_top))

    ffzne_mitigated_results = folding_free_zne(processed_results[0],job_param['N'],job_param['T'],job_param['system'],ffzne_top)

    best_ffzne_hellinger_fidelity = hellinger_fidelity(ffzne_mitigated_results,ideal_result)
    best_ffzne_tv = TV(ffzne_mitigated_results,ideal_result)

    distribution_data_df['FFZNE best distribution'] = distribution_data_df['State'].map(ffzne_mitigated_results)

    #RBZNE
    max_hellinger_fidelity = 0
    max_number_of_points = 0
    max_top = 0

    min_tv = 1
    min_number_of_points = 0
    min_top = 0

    for top in [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100]:
            for number_of_points in [2,3,4,5]:
                rbzne_mitigated_results = reliability_based_zne(processed_results,job_param['N'],job_param['T'],job_param['system'],number_of_points,top)
                print("Top: {}, Number of Points: {}, Hellinger Fidelity: {}, TV: {}".format(top,number_of_points,hellinger_fidelity(rbzne_mitigated_results,ideal_result),TV(rbzne_mitigated_results,ideal_result)))

                if hellinger_fidelity(rbzne_mitigated_results,ideal_result) > max_hellinger_fidelity:
                    max_hellinger_fidelity = hellinger_fidelity(rbzne_mitigated_results,ideal_result)
                    max_number_of_points, max_top = number_of_points, top

                if TV(rbzne_mitigated_results,ideal_result) < min_tv:
                    min_tv = TV(rbzne_mitigated_results,ideal_result)
                    min_number_of_points, min_top = number_of_points, top

    rbzne_top = max_top
    rbzne_number_of_points = max_number_of_points

    print()
    print("RBZNE completed")
    print("Max Hellinger Fidelity: {}, Number of Points: {}, Top: {}".format(max_hellinger_fidelity,max_number_of_points,max_top))
    print("Min TV: {}, Number of Points: {}, Top: {}".format(min_tv,min_number_of_points,min_top))

    rbzne_mitigated_results = reliability_based_zne(processed_results,job_param['N'],job_param['T'],job_param['system'],rbzne_number_of_points,rbzne_top)
    print("Hellinger fidelity: {} TV: {}".format(hellinger_fidelity(rbzne_mitigated_results,ideal_result),TV(rbzne_mitigated_results,ideal_result)))

    best_rbzne_hellinger_fidelity = hellinger_fidelity(rbzne_mitigated_results,ideal_result)
    best_rbzne_tv = TV(rbzne_mitigated_results,ideal_result)

    distribution_data_df['RBZNE best distribution'] = distribution_data_df['State'].map(rbzne_mitigated_results)

    #Sort based on Simulation probability
    distribution_data_df = distribution_data_df.sort_values(by='Simulation probability',ascending=False)

    counter = int(pow(2,job_param['N']) + 3)
    sheet_name = 'N_{}_T_{}'.format(job_param['N'],job_param['T'])

    summary_dict_list.append({'N': job_param['N'], 
                              'T': job_param['T'], 
                              'Initial Hellinger fidelity' : hellinger_fidelities[0],
                              'Initial TV' : TVs[0],
                              'ZNE Degree' : zne_degree,
                                'ZNE Number of points' : zne_number_of_points,
                                'ZNE Top k%' : zne_top,
                                'ZNE Hellinger fidelity' : best_zne_hellinger_fidelity,
                                'ZNE TV' : best_zne_tv,
                                'FFZNE Top k%' : ffzne_top,
                                'FFZNE Hellinger fidelity' : best_ffzne_hellinger_fidelity,
                                'FFZNE TV' : best_ffzne_tv,
                                'RBZNE Number of points' : rbzne_number_of_points,
                                'RBZNE Top k%' : rbzne_top,
                                'RBZNE Hellinger fidelity' : best_rbzne_hellinger_fidelity,
                                'RBZNE TV' : best_rbzne_tv
                                })

    #write in append mode
    with pd.ExcelWriter("../results/distribution_data.xlsx",mode='a', engine='openpyxl') as writer:
        distribution_data_df.to_excel(writer, sheet_name=sheet_name, index=False)
        workbook = writer.book
        worksheet = writer.sheets[sheet_name]
        
        # Accessing the active sheet
        active_sheet = workbook.active
        
        # Creating color scale rules
        color_scale_rule = ColorScaleRule(start_type='min', start_color='FF00FF00', end_type='max', end_color='FF00FF00')
        
        # Applying color scale rules to different ranges
        worksheet.conditional_formatting.add('H2:H{}'.format(int((zne_top * pow(2,job_param['N']) // 100) + 2)), color_scale_rule)
        worksheet.conditional_formatting.add('I2:I{}'.format(int((ffzne_top * pow(2,job_param['N']) // 100) + 2)), color_scale_rule)
        worksheet.conditional_formatting.add('J2:J{}'.format(int((rbzne_top * pow(2,job_param['N']) // 100) + 2)), color_scale_rule)

        # Write to cell A19
        worksheet['A{}'.format(str(counter))] = 'Hellinger fidelity'
        worksheet['A{}'.format(str(counter+1))] = 'TV'
        worksheet['C{}'.format(str(counter))] = initial_hellinger_fidelity
        worksheet['C{}'.format(str(counter+1))] = initial_TV
        worksheet['H{}'.format(str(counter))] = best_zne_hellinger_fidelity
        worksheet['H{}'.format(str(counter+1))] = best_zne_tv
        worksheet['I{}'.format(str(counter))] = best_ffzne_hellinger_fidelity
        worksheet['I{}'.format(str(counter+1))] = best_ffzne_tv
        worksheet['J{}'.format(str(counter))] = best_rbzne_hellinger_fidelity
        worksheet['J{}'.format(str(counter+1))] = best_rbzne_tv

        worksheet['A{}'.format(str(counter+3))] = 'Degree used for extrapolation'
        worksheet['A{}'.format(str(counter+4))] = 'Number of points used for extrapolation'
        worksheet['A{}'.format(str(counter+5))] = 'Top k percentage'
        worksheet['H{}'.format(str(counter+3))] = zne_degree
        worksheet['H{}'.format(str(counter+4))] = zne_number_of_points
        worksheet['H{}'.format(str(counter+5))] = zne_top
        worksheet['I{}'.format(str(counter+5))] = ffzne_top
        worksheet['J{}'.format(str(counter+4))] = rbzne_number_of_points
        worksheet['J{}'.format(str(counter+5))] = rbzne_top

        #Create a border around A19 to J20
        border = Border(left=Side(border_style='thin', color='FF000000'),
                        right=Side(border_style='thin', color='FF000000'),
                        top=Side(border_style='thin', color='FF000000'),
                        bottom=Side(border_style='thin', color='FF000000'))
        
        left_thick_border = Border(left=Side(border_style='medium', color='FF000000'))
        bottom_thick_border = Border(bottom=Side(border_style='medium', color='FF000000'))
        right_thick_border = Border(right=Side(border_style='medium', color='FF000000'))

        heading_border = Border(left=Side(border_style='medium', color='FF000000'),
                                right=Side(border_style='medium', color='FF000000'),
                                top=Side(border_style='medium', color='FF000000'),
                                bottom=Side(border_style='medium', color='FF000000'))
        
        bottom_right_thick_border = Border(right=Side(border_style='medium', color='FF000000'),
                                            bottom=Side(border_style='medium', color='FF000000'))

        for row in worksheet.iter_rows(min_row=1, max_row=counter-2, min_col=1, max_col=1):
            for cell in row:
                cell.border = left_thick_border

        for row in worksheet.iter_rows(min_row=1, max_row=counter-2, min_col=10, max_col=10):
            for cell in row:
                cell.border = right_thick_border

        for row in worksheet.iter_rows(min_row=counter-2, max_row=counter-2, min_col=1, max_col=10):
            for cell in row:
                cell.border = bottom_thick_border
        
        for row in worksheet.iter_rows(min_row=counter, max_row=counter+1, min_col=1, max_col=10):
            for cell in row:
                cell.border = border

        for row in worksheet.iter_rows(min_row=counter+3, max_row=counter+5, min_col=1, max_col=10):
            for cell in row:
                cell.border = border

        for row in worksheet.iter_rows(min_row=1, max_row=1, min_col=1, max_col=10):
            for cell in row:
                cell.border = heading_border

        #assign bottom right thick border to J19
        worksheet['J{}'.format(str(counter-2))].border = bottom_right_thick_border

        # Auto-adjusting column widths
        for column_cells in worksheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length

        # Centering text in every cell
        for row in worksheet.iter_rows():
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')

summary_df = pd.DataFrame(summary_dict_list)
with pd.ExcelWriter("../results/distribution_data.xlsx",mode='a', engine='openpyxl') as writer:
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
        workbook = writer.book
        worksheet = writer.sheets[sheet_name]
        
        # Accessing the active sheet
        active_sheet = workbook.active

        # Auto-adjusting column widths
        for column_cells in worksheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length

        # Centering text in every cell
        for row in worksheet.iter_rows():
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')


In [33]:
#Getting the job params and the layout that was used
job_param = job_params[0]
job_param

{'system': 'ibm_brisbane', 'N': 4, 'T': 1, 'job_id': 'cq65fkp28rp0008y0tg0'}

In [34]:
circuit_filename = "../circuits/" + job_param['system'] + "_" + str(job_param['N']) + "_" + str(job_param['T']) + ".pkl"
with open(circuit_filename, 'rb') as f:
    circuit = pickle.load(f)
layout = circuit['layout']
layout

[0, 1, 2, 3]

In [35]:
#Getting the ideal results
ideal_result = ideal_results[job_param['N'],job_param['T']]

#Getting the raw results
raw_results = provider.retrieve_job(job_param['job_id']).result().get_counts()
processed_results = list(map(lambda x: process_results(x,layout),raw_results))

keys = set(ideal_result.keys()) | set(processed_results[0].keys()) | set(processed_results[1].keys()) | set(processed_results[2].keys()) | set(processed_results[3].keys()) | set(processed_results[4].keys())
keys = sorted(list(keys))

#Create a df and assign keys as index and name the column as State and add the values of keys to that column
distribution_data_df = pd.DataFrame(keys,columns=['State'],index=keys)


In [ ]:
#Map the ideal results to the dataframe
distribution_data_df['Ideal probability'] = distribution_data_df['State'].map(ideal_result)
distribution_data_df

In [ ]:
#Getting the processed results
distribution_data_df['Simulation probability'] = distribution_data_df['State'].map(processed_results[0])
distribution_data_df['Noise factor 3'] = distribution_data_df['State'].map(processed_results[1])
distribution_data_df['Noise factor 5'] = distribution_data_df['State'].map(processed_results[2])
distribution_data_df['Noise factor 7'] = distribution_data_df['State'].map(processed_results[3])
distribution_data_df['Noise factor 9'] = distribution_data_df['State'].map(processed_results[4])
distribution_data_df

In [ ]:
#Hellinger fidelity
hellinger_fidelities = list(map(lambda x: hellinger_fidelity(ideal_result,x),processed_results))
initial_hellinger_fidelity = hellinger_fidelities[0]
hellinger_fidelities

In [41]:
#Total variational distance
TVs = list(map(lambda x: TV(ideal_result,x),processed_results))
initial_TV = TVs[0]
TVs

[0.29839976004521235,
 0.3740697882299207,
 0.41128887343848936,
 0.40005840468848936,
 0.3249256123356375]

In [ ]:
#ZNE
max_hellinger_fidelity = 0
max_number_of_points = 0
max_degree = 0
max_top = 0

min_tv = 1
min_number_of_points = 0
min_degree = 0
min_top = 0

for top in [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100]:
    for degree in [1,2]:
        for number_of_points in [2,3,4,5]:
            zne_processed_results = zne(processed_results,degree,number_of_points,top)
            print("Top: {}, Degree: {}, Number of Points: {}, Hellinger Fidelity: {}, TV: {}".format(top,degree,number_of_points,hellinger_fidelity(zne_processed_results,ideal_result),TV(zne_processed_results,ideal_result)))

            if hellinger_fidelity(zne_processed_results,ideal_result) > max_hellinger_fidelity:
                max_hellinger_fidelity = hellinger_fidelity(zne_processed_results,ideal_result)
                max_number_of_points, max_degree, max_top = number_of_points, degree, top

            if TV(zne_processed_results,ideal_result) < min_tv:
                min_tv = TV(zne_processed_results,ideal_result)
                min_number_of_points, min_degree, min_top = number_of_points, degree, top

zne_degree = max_degree
zne_number_of_points = max_number_of_points
zne_top = max_top

print("ZNE completed")
print("Max Hellinger Fidelity: {}, Number of Points: {}, Degree: {}, Top: {}".format(max_hellinger_fidelity,max_number_of_points,max_degree,max_top))
print("Min TV: {}, Number of Points: {}, Degree: {}, Top: {}".format(min_tv,min_number_of_points,min_degree,min_top))

In [43]:
zne_processed_results = zne(processed_results,zne_degree,zne_number_of_points,zne_top)

best_zne_hellinger_fidelity = hellinger_fidelity(zne_processed_results,ideal_result)
best_zne_tv = TV(zne_processed_results,ideal_result)

print("Hellinger fidelity: {} TV: {}".format(hellinger_fidelity(zne_processed_results,ideal_result),TV(zne_processed_results,ideal_result)))

Hellinger fidelity: 0.8783043803872781 TV: 0.2858136194730446


In [ ]:
distribution_data_df['ZNE best distribution'] = distribution_data_df['State'].map(zne_processed_results)
distribution_data_df

In [ ]:
#ffzne
max_hellinger_fidelity = 0
max_top = 0

min_tv = 1
min_top = 0

for top in [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100]:
    ffzne_mitigated_results = folding_free_zne(processed_results[0],job_param['N'],job_param['T'],job_param['system'],top)
    print("Top: {}, Hellinger Fidelity: {}, TV: {}".format(top,hellinger_fidelity(ffzne_mitigated_results,ideal_result),TV(ffzne_mitigated_results,ideal_result)))

    if hellinger_fidelity(ffzne_mitigated_results,ideal_result) > max_hellinger_fidelity:
        max_hellinger_fidelity = hellinger_fidelity(ffzne_mitigated_results,ideal_result)
        max_top = top

    if TV(ffzne_mitigated_results,ideal_result) < min_tv:
        min_tv = TV(ffzne_mitigated_results,ideal_result)
        min_top = top

ffzne_top = max_top

print("FFZNE completed")
print("Max Hellinger Fidelity: {}, Top: {}".format(max_hellinger_fidelity,max_top))
print("Min TV: {}, Top: {}".format(min_tv,min_top))

In [ ]:
ffzne_mitigated_results = folding_free_zne(processed_results[0],job_param['N'],job_param['T'],job_param['system'],ffzne_top)

best_ffzne_hellinger_fidelity = hellinger_fidelity(ffzne_mitigated_results,ideal_result)
best_ffzne_tv = TV(ffzne_mitigated_results,ideal_result)

distribution_data_df['FFZNE best distribution'] = distribution_data_df['State'].map(ffzne_mitigated_results)
distribution_data_df

In [ ]:
#RBZNE
max_hellinger_fidelity = 0
max_number_of_points = 0
max_top = 0

min_tv = 1
min_number_of_points = 0
min_top = 0

for top in [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100]:
        for number_of_points in [2,3,4,5]:
            rbzne_mitigated_results = reliability_based_zne(processed_results,job_param['N'],job_param['T'],job_param['system'],number_of_points,top)
            print("Top: {}, Number of Points: {}, Hellinger Fidelity: {}, TV: {}".format(top,number_of_points,hellinger_fidelity(rbzne_mitigated_results,ideal_result),TV(rbzne_mitigated_results,ideal_result)))

            if hellinger_fidelity(rbzne_mitigated_results,ideal_result) > max_hellinger_fidelity:
                max_hellinger_fidelity = hellinger_fidelity(rbzne_mitigated_results,ideal_result)
                max_number_of_points, max_top = number_of_points, top

            if TV(rbzne_mitigated_results,ideal_result) < min_tv:
                min_tv = TV(rbzne_mitigated_results,ideal_result)
                min_number_of_points, min_top = number_of_points, top

rbzne_top = max_top
rbzne_number_of_points = max_number_of_points

print()
print("RBZNE completed")
print("Max Hellinger Fidelity: {}, Number of Points: {}, Top: {}".format(max_hellinger_fidelity,max_number_of_points,max_top))
print("Min TV: {}, Number of Points: {}, Top: {}".format(min_tv,min_number_of_points,min_top))

In [ ]:
rbzne_mitigated_results = reliability_based_zne(processed_results,job_param['N'],job_param['T'],job_param['system'],rbzne_number_of_points,rbzne_top)
print("Hellinger fidelity: {} TV: {}".format(hellinger_fidelity(rbzne_mitigated_results,ideal_result),TV(rbzne_mitigated_results,ideal_result)))

best_rbzne_hellinger_fidelity = hellinger_fidelity(rbzne_mitigated_results,ideal_result)
best_rbzne_tv = TV(rbzne_mitigated_results,ideal_result)

distribution_data_df['RBZNE best distribution'] = distribution_data_df['State'].map(rbzne_mitigated_results)
distribution_data_df

In [ ]:
#Sort based on Simulation probability
distribution_data_df = distribution_data_df.sort_values(by='Simulation probability',ascending=False)
distribution_data_df

In [56]:
import pandas as pd
from openpyxl.formatting.rule import ColorScaleRule
from math import pow
from openpyxl.styles import Alignment
from openpyxl.styles import Border, Side

counter = int(pow(2,job_param['N']) + 3)
sheet_name = 'N_{}_T_{}'.format(job_param['N'],job_param['T'])

with pd.ExcelWriter("../results/distribution_data.xlsx",mode='a', engine='openpyxl') as writer:
    distribution_data_df.to_excel(writer, sheet_name=sheet_name, index=False)
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    
    # Accessing the active sheet
    active_sheet = workbook.active
    
    # Creating color scale rules
    color_scale_rule = ColorScaleRule(start_type='min', start_color='FF00FF00', end_type='max', end_color='FF00FF00')
    
    # Applying color scale rules to different ranges
    worksheet.conditional_formatting.add('H2:H{}'.format(int((zne_top * pow(2,job_param['N']) // 100) + 2)), color_scale_rule)
    worksheet.conditional_formatting.add('I2:I{}'.format(int((ffzne_top * pow(2,job_param['N']) // 100) + 2)), color_scale_rule)
    worksheet.conditional_formatting.add('J2:J{}'.format(int((rbzne_top * pow(2,job_param['N']) // 100) + 2)), color_scale_rule)

    # Write to cell A19
    worksheet['A{}'.format(str(counter))] = 'Hellinger fidelity'
    worksheet['A{}'.format(str(counter+1))] = 'TV'
    worksheet['C{}'.format(str(counter))] = initial_hellinger_fidelity
    worksheet['C{}'.format(str(counter+1))] = initial_TV
    worksheet['H{}'.format(str(counter))] = best_zne_hellinger_fidelity
    worksheet['H{}'.format(str(counter+1))] = best_zne_tv
    worksheet['I{}'.format(str(counter))] = best_ffzne_hellinger_fidelity
    worksheet['I{}'.format(str(counter+1))] = best_ffzne_tv
    worksheet['J{}'.format(str(counter))] = best_rbzne_hellinger_fidelity
    worksheet['J{}'.format(str(counter+1))] = best_rbzne_tv

    worksheet['A{}'.format(str(counter+3))] = 'Degree used for extrapolation'
    worksheet['A{}'.format(str(counter+4))] = 'Number of points used for extrapolation'
    worksheet['H{}'.format(str(counter+3))] = zne_degree
    worksheet['H{}'.format(str(counter+4))] = zne_number_of_points
    worksheet['J{}'.format(str(counter+4))] = rbzne_number_of_points

    #Create a border around A19 to J20
    border = Border(left=Side(border_style='thin', color='FF000000'),
                    right=Side(border_style='thin', color='FF000000'),
                    top=Side(border_style='thin', color='FF000000'),
                    bottom=Side(border_style='thin', color='FF000000'))
    
    left_thick_border = Border(left=Side(border_style='medium', color='FF000000'))
    bottom_thick_border = Border(bottom=Side(border_style='medium', color='FF000000'))
    right_thick_border = Border(right=Side(border_style='medium', color='FF000000'))

    heading_border = Border(left=Side(border_style='medium', color='FF000000'),
                            right=Side(border_style='medium', color='FF000000'),
                            top=Side(border_style='medium', color='FF000000'),
                            bottom=Side(border_style='medium', color='FF000000'))
    
    bottom_right_thick_border = Border(right=Side(border_style='medium', color='FF000000'),
                                        bottom=Side(border_style='medium', color='FF000000'))

    for row in worksheet.iter_rows(min_row=1, max_row=counter-2, min_col=1, max_col=1):
        for cell in row:
            cell.border = left_thick_border

    for row in worksheet.iter_rows(min_row=1, max_row=counter-2, min_col=10, max_col=10):
        for cell in row:
            cell.border = right_thick_border

    for row in worksheet.iter_rows(min_row=counter-2, max_row=counter-2, min_col=1, max_col=10):
        for cell in row:
            cell.border = bottom_thick_border
    
    for row in worksheet.iter_rows(min_row=counter, max_row=counter+1, min_col=1, max_col=10):
        for cell in row:
            cell.border = border

    for row in worksheet.iter_rows(min_row=counter+3, max_row=counter+4, min_col=1, max_col=10):
        for cell in row:
            cell.border = border

    for row in worksheet.iter_rows(min_row=1, max_row=1, min_col=1, max_col=10):
        for cell in row:
            cell.border = heading_border

    #assign bottom right thick border to J19
    worksheet['J{}'.format(str(counter-2))].border = bottom_right_thick_border

    # Auto-adjusting column widths
    for column_cells in worksheet.columns:
        length = max(len(str(cell.value)) for cell in column_cells)
        worksheet.column_dimensions[column_cells[0].column_letter].width = length

    # Centering text in every cell
    for row in worksheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(horizontal='center', vertical='center')

### Getting the results for Top 10

#### Run on qiskit backend

In [28]:
summary_dict_list = []

for system in systems:
    
    distribution_data_df_list = []

    print("Processing system: {}".format(system))
    backend = provider.get_backend(system)

    system_job_params = list(filter(lambda x: x['system'] == system,job_params))
    print("Number of jobs: {}".format(len(system_job_params)))
    
    for job_param in system_job_params:
        print("Processing N: {}, T: {}".format(job_param['N'],job_param['T']))
        circuit_filename = "../circuits/" + job_param['system'] + "_" + str(job_param['N']) + "_" + str(job_param['T']) + ".pkl"
        with open(circuit_filename, 'rb') as f:
            circuit = pickle.load(f)

        reliabilities = circuit['circuit_reliabilities']
        layout = circuit['layout']

        circuit_reliability_filename = "../circuits/modified/" + job_param['system'] + "_" + str(job_param['N']) + "_" + str(job_param['T']) + "_reliabilities.pkl"
        with open(circuit_reliability_filename, 'rb') as f:
            circuit_reliabilities = pickle.load(f)['circuit_reliabilities']

        #Getting the ideal results
        ideal_result = ideal_results[job_param['N'],job_param['T']]
        #without_noise_simulation_result = simulate(circuit['circuit'][0], backend, with_noise=False)

        #Getting the raw results
        raw_results = provider.retrieve_job(job_param['job_id']).result().get_counts()
        processed_results = list(map(lambda x: process_results(x,layout),raw_results))

        keys = set(ideal_result.keys()) | \
                set(processed_results[0].keys()) | \
                set(processed_results[1].keys()) | \
                set(processed_results[2].keys()) | \
                set(processed_results[3].keys()) | \
                set(processed_results[4].keys())
        
        keys = sorted(list(keys))

        #Create a df and assign keys as index and name the column as State and add the values of keys to that column
        distribution_data_df = pd.DataFrame(keys,columns=['State'],index=keys)

        #Map the results to the dataframe
        distribution_data_df['Ideal probability'] = distribution_data_df['State'].map(ideal_result)
        #distribution_data_df['Without Noise Simulation probability'] = distribution_data_df['State'].map(without_noise_simulation_result)
        distribution_data_df['Noise factor 1 probability'] = distribution_data_df['State'].map(processed_results[0])
        distribution_data_df['Noise factor 3 probability'] = distribution_data_df['State'].map(processed_results[1])
        distribution_data_df['Noise factor 5 probability'] = distribution_data_df['State'].map(processed_results[2])
        distribution_data_df['Noise factor 7 probability'] = distribution_data_df['State'].map(processed_results[3])
        distribution_data_df['Noise factor 9 probability'] = distribution_data_df['State'].map(processed_results[4])

        hellinger_fidelities = list(map(lambda x: hellinger_fidelity(ideal_result,x),processed_results))
        #without_noise_hellinger_fidelity = hellinger_fidelity(ideal_result,without_noise_simulation_result)
        initial_hellinger_fidelity = hellinger_fidelities[0]

        #without_noise_TV = TV(ideal_result,without_noise_simulation_result)
        TVs = list(map(lambda x: TV(ideal_result,x),processed_results))
        initial_TV = TVs[0]

        #ZNE
        zne_processed_results = zne(processed_results,2,3,10)

        best_zne_hellinger_fidelity = hellinger_fidelity(zne_processed_results,ideal_result)
        best_zne_tv = TV(zne_processed_results,ideal_result)
        distribution_data_df['ZNE probability distribution'] = distribution_data_df['State'].map(zne_processed_results)

        #ffzne
        ffzne_mitigated_results = folding_free_zne(processed_results[0],reliabilities[0],job_param['N'],10)
        best_ffzne_hellinger_fidelity = hellinger_fidelity(ffzne_mitigated_results,ideal_result)
        best_ffzne_tv = TV(ffzne_mitigated_results,ideal_result)
        distribution_data_df['FFZNE probability distribution'] = distribution_data_df['State'].map(ffzne_mitigated_results)

        #RBZNE
        rbzne_mitigated_results = reliability_based_zne(processed_results,job_param['N'],reliabilities,2,10)
        best_rbzne_hellinger_fidelity = hellinger_fidelity(rbzne_mitigated_results,ideal_result)
        best_rbzne_tv = TV(rbzne_mitigated_results,ideal_result)
        distribution_data_df['RBZNE probability distribution'] = distribution_data_df['State'].map(rbzne_mitigated_results)

        #Sort based on Simulation probability
        distribution_data_df = distribution_data_df.sort_values(by='Noise factor 1 probability',ascending=False)

        counter = int(pow(2,job_param['N']) + 3)
        sheet_name = 'N_{}_T_{}'.format(job_param['N'],job_param['T'])

        summary_dict_list.append({  'N': job_param['N'], 
                                    'T': job_param['T'], 
                                    'System': job_param['system'],
                                    #'Without Noise HF' : without_noise_hellinger_fidelity,
                                    'Noise HF' : hellinger_fidelities[0],
                                    'ZNE HF' : best_zne_hellinger_fidelity,
                                    'FFZNE HF' : best_ffzne_hellinger_fidelity,
                                    'RBZNE HF' : best_rbzne_hellinger_fidelity,
                                    #'Without Noise TV' : without_noise_TV,
                                    'Noise TV' : TVs[0],
                                    'ZNE TV' : best_zne_tv,
                                    'FFZNE TV' : best_ffzne_tv,
                                    'RBZNE TV' : best_rbzne_tv
                                })

        #Save the distribution_data_df to pickle
        distribution_data_df_list.append(distribution_data_df)

        #write in append mode
        with pd.ExcelWriter("../results/{}_distribution_data.xlsx".format(system), mode='a', engine='openpyxl') as writer:
            distribution_data_df.to_excel(writer, sheet_name=sheet_name, index=False)
            workbook = writer.book
            worksheet = writer.sheets[sheet_name]
            
            # Accessing the active sheet
            active_sheet = workbook.active
            
            # Write to cell A19
            worksheet['A{}'.format(str(counter))] = 'Hellinger fidelity'
            worksheet['A{}'.format(str(counter+1))] = 'TV'
            worksheet['C{}'.format(str(counter))] = initial_hellinger_fidelity
            worksheet['C{}'.format(str(counter+1))] = initial_TV
            worksheet['H{}'.format(str(counter))] = best_zne_hellinger_fidelity
            worksheet['H{}'.format(str(counter+1))] = best_zne_tv
            worksheet['I{}'.format(str(counter))] = best_ffzne_hellinger_fidelity
            worksheet['I{}'.format(str(counter+1))] = best_ffzne_tv
            worksheet['J{}'.format(str(counter))] = best_rbzne_hellinger_fidelity
            worksheet['J{}'.format(str(counter+1))] = best_rbzne_tv

            worksheet['A{}'.format(str(counter+2))] = 'Circuit reliabilities'
            worksheet['C{}'.format(str(counter+2))] = circuit_reliabilities[0]
            worksheet['D{}'.format(str(counter+2))] = circuit_reliabilities[1]
            worksheet['E{}'.format(str(counter+2))] = circuit_reliabilities[2]
            worksheet['F{}'.format(str(counter+2))] = circuit_reliabilities[3]
            worksheet['G{}'.format(str(counter+2))] = circuit_reliabilities[4]

            worksheet['A{}'.format(str(counter+3))] = 'Circuit depth'
            worksheet['C{}'.format(str(counter+3))] = circuit['circuit'][0].depth()
            worksheet['D{}'.format(str(counter+3))] = circuit['circuit'][1].depth()
            worksheet['E{}'.format(str(counter+3))] = circuit['circuit'][2].depth()
            worksheet['F{}'.format(str(counter+3))] = circuit['circuit'][3].depth()
            worksheet['G{}'.format(str(counter+3))] = circuit['circuit'][4].depth()

            worksheet['A{}'.format(str(counter+4))] = 'RZX count'
            worksheet['C{}'.format(str(counter+4))] = circuit['circuit'][0].count_ops()['rzx']
            worksheet['D{}'.format(str(counter+4))] = circuit['circuit'][1].count_ops()['rzx']
            worksheet['E{}'.format(str(counter+4))] = circuit['circuit'][2].count_ops()['rzx']
            worksheet['F{}'.format(str(counter+4))] = circuit['circuit'][3].count_ops()['rzx']
            worksheet['G{}'.format(str(counter+4))] = circuit['circuit'][4].count_ops()['rzx']

            worksheet['A{}'.format(str(counter+5))] = 'RX count'
            worksheet['C{}'.format(str(counter+5))] = circuit['circuit'][0].count_ops()['rx']
            worksheet['D{}'.format(str(counter+5))] = circuit['circuit'][1].count_ops()['rx']
            worksheet['E{}'.format(str(counter+5))] = circuit['circuit'][2].count_ops()['rx']
            worksheet['F{}'.format(str(counter+5))] = circuit['circuit'][3].count_ops()['rx']
            worksheet['G{}'.format(str(counter+5))] = circuit['circuit'][4].count_ops()['rx']

            #Create a border around A19 to J21
            border = Border(left=Side(border_style='thin', color='FF000000'),
                            right=Side(border_style='thin', color='FF000000'),
                            top=Side(border_style='thin', color='FF000000'),
                            bottom=Side(border_style='thin', color='FF000000'))
            
            left_thick_border = Border(left=Side(border_style='medium', color='FF000000'))
            bottom_thick_border = Border(bottom=Side(border_style='medium', color='FF000000'))
            right_thick_border = Border(right=Side(border_style='medium', color='FF000000'))

            heading_border = Border(left=Side(border_style='medium', color='FF000000'),
                                    right=Side(border_style='medium', color='FF000000'),
                                    top=Side(border_style='medium', color='FF000000'),
                                    bottom=Side(border_style='medium', color='FF000000'))
            
            bottom_right_thick_border = Border(right=Side(border_style='medium', color='FF000000'),
                                                bottom=Side(border_style='medium', color='FF000000'))

            for row in worksheet.iter_rows(min_row=1, max_row=counter-2, min_col=1, max_col=1):
                for cell in row:
                    cell.border = left_thick_border

            for row in worksheet.iter_rows(min_row=1, max_row=counter-2, min_col=10, max_col=10):
                for cell in row:
                    cell.border = right_thick_border

            for row in worksheet.iter_rows(min_row=counter-2, max_row=counter-2, min_col=1, max_col=10):
                for cell in row:
                    cell.border = bottom_thick_border
            
            for row in worksheet.iter_rows(min_row=counter, max_row=counter+5, min_col=1, max_col=10):
                for cell in row:
                    cell.border = border

            for row in worksheet.iter_rows(min_row=1, max_row=1, min_col=1, max_col=10):
                for cell in row:
                    cell.border = heading_border

            #assign bottom right thick border to J19
            worksheet['J{}'.format(str(counter-2))].border = bottom_right_thick_border

            # Auto-adjusting column widths
            for column_cells in worksheet.columns:
                length = max(len(str(cell.value)) for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = length

            # Centering text in every cell
            for row in worksheet.iter_rows():
                for cell in row:
                    cell.alignment = Alignment(horizontal='center', vertical='center')

    #Pickle
    with open("../results/{}_distribution_data.pkl".format(system), 'wb') as f:
        pickle.dump(distribution_data_df_list, f)

    #break


#Running for parameteric RBZNE with extra point
#RBZNE for parameteric and without extra point gave good result

Processing system: ibm_sherbrooke
Number of jobs: 48
Processing N: 4, T: 0.1
Processing N: 4, T: 0.2
Processing N: 4, T: 0.3
Processing N: 4, T: 0.4
Processing N: 4, T: 0.5
Processing N: 4, T: 0.6
Processing N: 4, T: 0.7
Processing N: 4, T: 0.8
Processing N: 4, T: 0.9
Processing N: 4, T: 1.0
Processing N: 4, T: 1.5
Processing N: 4, T: 2.0
Processing N: 6, T: 0.1
Processing N: 6, T: 0.2
Processing N: 6, T: 0.3
Processing N: 6, T: 0.4
Processing N: 6, T: 0.5
Processing N: 6, T: 0.6
Processing N: 6, T: 0.7
Processing N: 6, T: 0.8
Processing N: 6, T: 0.9
Processing N: 6, T: 1.0
Processing N: 6, T: 1.5
Processing N: 6, T: 2.0
Processing N: 8, T: 0.1
Processing N: 8, T: 0.2
Processing N: 8, T: 0.3
Processing N: 8, T: 0.4
Processing N: 8, T: 0.5
Processing N: 8, T: 0.6
Processing N: 8, T: 0.7
Processing N: 8, T: 0.8
Processing N: 8, T: 0.9
Processing N: 8, T: 1.0
Processing N: 8, T: 1.5
Processing N: 8, T: 2.0
Processing N: 10, T: 0.1
Processing N: 10, T: 0.2
Processing N: 10, T: 0.3
Processi

In [ ]:
summary_df = pd.DataFrame(summary_dict_list)

summary_df['ZNE HF % change'] = (summary_df['ZNE HF'] - summary_df['Noise HF']) / summary_df['Noise HF'] * 100
summary_df['FFZNE HF % change'] = (summary_df['FFZNE HF'] - summary_df['Noise HF']) / summary_df['Noise HF'] * 100
summary_df['RBZNE HF % change'] = (summary_df['RBZNE HF'] - summary_df['Noise HF']) / summary_df['Noise HF'] * 100
summary_df['ZNE TV % change'] = (summary_df['Noise TV'] - summary_df['ZNE TV']) / summary_df['Noise TV'] * 100
summary_df['FFZNE TV % change'] = (summary_df['Noise TV'] - summary_df['FFZNE TV']) / summary_df['Noise TV'] * 100
summary_df['RBZNE TV % change'] = (summary_df['Noise TV'] - summary_df['RBZNE TV']) / summary_df['Noise TV'] * 100

summary_df = summary_df[[   'N',
                            'T',
                            'System',
                            'Noise HF',
                            'ZNE HF',
                            'ZNE HF % change',
                            'FFZNE HF',
                            'FFZNE HF % change',
                            'RBZNE HF',
                            'RBZNE HF % change',
                            'Noise TV',
                            'ZNE TV',
                            'ZNE TV % change',
                            'FFZNE TV',
                            'FFZNE TV % change',
                            'RBZNE TV',
                            'RBZNE TV % change'
                        ]]

                         
# Pickle the summary_df
summary_df.to_pickle("../results/summary_df.pkl")

sheet_name = 'Summary'

with pd.ExcelWriter("../results/summary_data.xlsx".format(system), engine='openpyxl') as writer:
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
        workbook = writer.book
        worksheet = writer.sheets[sheet_name]
        
        # Accessing the active sheet
        active_sheet = workbook.active

        # Calculate Geometric mean of 1 + (% change / 100) for Hellinger fidelity and TV
        worksheet['A195'] = 'Geometric mean'
        worksheet['F195'] = (1 + summary_df['ZNE HF % change'] / 100).prod() ** (1 / len(summary_df))
        worksheet['H195'] = (1 + summary_df['FFZNE HF % change'] / 100).prod() ** (1 / len(summary_df))
        worksheet['J195'] = (1 + summary_df['RBZNE HF % change'] / 100).prod() ** (1 / len(summary_df))
        worksheet['M195'] = (1 + summary_df['ZNE TV % change'] / 100).prod() ** (1 / len(summary_df))
        worksheet['O195'] = (1 + summary_df['FFZNE TV % change'] / 100).prod() ** (1 / len(summary_df))
        worksheet['Q195'] = (1 + summary_df['RBZNE TV % change'] / 100).prod() ** (1 / len(summary_df))

        # Auto-adjusting column widths
        for column_cells in worksheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length

        # Centering text in every cell
        for row in worksheet.iter_rows():
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')

#### Run on Sampler

In [12]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.result import QuasiDistribution

service = QiskitRuntimeService(channel="ibm_quantum")

for system in systems:
    print("Processing system: {}".format(system))

    system_job_params = list(filter(lambda x: x['system'] == system,job_params_sampler))
    print("Number of jobs: {}".format(len(system_job_params)))
    summary_dict_list = []

    for job_param in system_job_params:
        print("Processing N: {}, T: {}".format(job_param['N'],job_param['T']))
        circuit_filename = "../circuits/" + job_param['system'] + "_" + str(job_param['N']) + "_" + str(job_param['T']) + ".pkl"
        with open(circuit_filename, 'rb') as f:
            circuit = pickle.load(f)
        layout = circuit['layout']

        circuit_reliability_filename = "../circuits/modified/" + job_param['system'] + "_" + str(job_param['N']) + "_" + str(job_param['T']) + "_reliabilities.pkl"
        with open(circuit_reliability_filename, 'rb') as f:
            circuit_reliabilities = pickle.load(f)['circuit_reliabilities']

        #Getting the ideal results
        ideal_result = ideal_results[job_param['N'],job_param['T']]

        #Getting the raw results
        quasi_dists = service.job(job_param['job_id']).result().quasi_dists
        quasi_dists = QuasiDistribution(result.quasi_dists[0],shots=1024)
        quasi_dists.nearest_probability_distribution()
        binary_key_lambda = lambda x: bin(x)[2:].zfill(127)
        binary_keys = map(binary_key_lambda, quasi_dists.keys())

        raw_result = dict(zip(binary_keys, quasi_dists.values()))


        processed_results = list(map(lambda x: process_results(x,layout),raw_results))

        keys = set(ideal_result.keys()) | set(processed_results[0].keys()) | set(processed_results[1].keys()) | set(processed_results[2].keys()) | set(processed_results[3].keys()) | set(processed_results[4].keys())
        keys = sorted(list(keys))

        #Create a df and assign keys as index and name the column as State and add the values of keys to that column
        distribution_data_df = pd.DataFrame(keys,columns=['State'],index=keys)

        #Map the results to the dataframe
        distribution_data_df['Ideal probability'] = distribution_data_df['State'].map(ideal_result)
        distribution_data_df['Simulation probability'] = distribution_data_df['State'].map(processed_results[0])
        distribution_data_df['Noise factor 3'] = distribution_data_df['State'].map(processed_results[1])
        distribution_data_df['Noise factor 5'] = distribution_data_df['State'].map(processed_results[2])
        distribution_data_df['Noise factor 7'] = distribution_data_df['State'].map(processed_results[3])
        distribution_data_df['Noise factor 9'] = distribution_data_df['State'].map(processed_results[4])

        hellinger_fidelities = list(map(lambda x: hellinger_fidelity(ideal_result,x),processed_results))
        initial_hellinger_fidelity = hellinger_fidelities[0]

        TVs = list(map(lambda x: TV(ideal_result,x),processed_results))
        initial_TV = TVs[0]

        #ZNE
        zne_processed_results = zne(processed_results,2,3,10)

        best_zne_hellinger_fidelity = hellinger_fidelity(zne_processed_results,ideal_result)
        best_zne_tv = TV(zne_processed_results,ideal_result)
        distribution_data_df['ZNE best distribution'] = distribution_data_df['State'].map(zne_processed_results)

        #ffzne
        ffzne_mitigated_results = folding_free_zne(processed_results[0],job_param['N'],job_param['T'],job_param['system'],10)
        best_ffzne_hellinger_fidelity = hellinger_fidelity(ffzne_mitigated_results,ideal_result)
        best_ffzne_tv = TV(ffzne_mitigated_results,ideal_result)
        distribution_data_df['FFZNE best distribution'] = distribution_data_df['State'].map(ffzne_mitigated_results)

        #RBZNE
        rbzne_mitigated_results = reliability_based_zne(processed_results,job_param['N'],job_param['T'],job_param['system'],3,10)
        best_rbzne_hellinger_fidelity = hellinger_fidelity(rbzne_mitigated_results,ideal_result)
        best_rbzne_tv = TV(rbzne_mitigated_results,ideal_result)
        distribution_data_df['RBZNE best distribution'] = distribution_data_df['State'].map(rbzne_mitigated_results)

        #Sort based on Simulation probability
        distribution_data_df = distribution_data_df.sort_values(by='Simulation probability',ascending=False)

        counter = int(pow(2,job_param['N']) + 3)
        sheet_name = 'N_{}_T_{}'.format(job_param['N'],job_param['T'])

        summary_dict_list.append({'N': job_param['N'], 
                                'T': job_param['T'], 
                                'Initial Hellinger fidelity' : hellinger_fidelities[0],
                                'Initial TV' : TVs[0],
                                    'ZNE Hellinger fidelity' : best_zne_hellinger_fidelity,
                                    'ZNE TV' : best_zne_tv,
                                    'FFZNE Hellinger fidelity' : best_ffzne_hellinger_fidelity,
                                    'FFZNE TV' : best_ffzne_tv,
                                    'RBZNE Hellinger fidelity' : best_rbzne_hellinger_fidelity,
                                    'RBZNE TV' : best_rbzne_tv
                                    })

        #write in append mode
        with pd.ExcelWriter("../results/{}_distribution_data.xlsx".format(system),mode='a', engine='openpyxl') as writer:
            distribution_data_df.to_excel(writer, sheet_name=sheet_name, index=False)
            workbook = writer.book
            worksheet = writer.sheets[sheet_name]
            
            # Accessing the active sheet
            active_sheet = workbook.active
            
            # Write to cell A19
            worksheet['A{}'.format(str(counter))] = 'Hellinger fidelity'
            worksheet['A{}'.format(str(counter+1))] = 'TV'
            worksheet['C{}'.format(str(counter))] = initial_hellinger_fidelity
            worksheet['C{}'.format(str(counter+1))] = initial_TV
            worksheet['H{}'.format(str(counter))] = best_zne_hellinger_fidelity
            worksheet['H{}'.format(str(counter+1))] = best_zne_tv
            worksheet['I{}'.format(str(counter))] = best_ffzne_hellinger_fidelity
            worksheet['I{}'.format(str(counter+1))] = best_ffzne_tv
            worksheet['J{}'.format(str(counter))] = best_rbzne_hellinger_fidelity
            worksheet['J{}'.format(str(counter+1))] = best_rbzne_tv

            worksheet['A{}'.format(str(counter+2))] = 'Circuit reliabilities'
            worksheet['C{}'.format(str(counter+2))] = circuit_reliabilities[0]
            worksheet['D{}'.format(str(counter+2))] = circuit_reliabilities[1]
            worksheet['E{}'.format(str(counter+2))] = circuit_reliabilities[2]
            worksheet['F{}'.format(str(counter+2))] = circuit_reliabilities[3]
            worksheet['G{}'.format(str(counter+2))] = circuit_reliabilities[4]

            worksheet['A{}'.format(str(counter+3))] = 'Circuit depth'
            worksheet['C{}'.format(str(counter+3))] = circuit['circuit'][0].depth()
            worksheet['D{}'.format(str(counter+3))] = circuit['circuit'][1].depth()
            worksheet['E{}'.format(str(counter+3))] = circuit['circuit'][2].depth()
            worksheet['F{}'.format(str(counter+3))] = circuit['circuit'][3].depth()
            worksheet['G{}'.format(str(counter+3))] = circuit['circuit'][4].depth()

            worksheet['A{}'.format(str(counter+4))] = 'RZX count'
            worksheet['C{}'.format(str(counter+4))] = circuit['circuit'][0].count_ops()['rzx']
            worksheet['D{}'.format(str(counter+4))] = circuit['circuit'][1].count_ops()['rzx']
            worksheet['E{}'.format(str(counter+4))] = circuit['circuit'][2].count_ops()['rzx']
            worksheet['F{}'.format(str(counter+4))] = circuit['circuit'][3].count_ops()['rzx']
            worksheet['G{}'.format(str(counter+4))] = circuit['circuit'][4].count_ops()['rzx']

            worksheet['A{}'.format(str(counter+5))] = 'RX count'
            worksheet['C{}'.format(str(counter+5))] = circuit['circuit'][0].count_ops()['rx']
            worksheet['D{}'.format(str(counter+5))] = circuit['circuit'][1].count_ops()['rx']
            worksheet['E{}'.format(str(counter+5))] = circuit['circuit'][2].count_ops()['rx']
            worksheet['F{}'.format(str(counter+5))] = circuit['circuit'][3].count_ops()['rx']
            worksheet['G{}'.format(str(counter+5))] = circuit['circuit'][4].count_ops()['rx']

            #Create a border around A19 to J21
            border = Border(left=Side(border_style='thin', color='FF000000'),
                            right=Side(border_style='thin', color='FF000000'),
                            top=Side(border_style='thin', color='FF000000'),
                            bottom=Side(border_style='thin', color='FF000000'))
            
            left_thick_border = Border(left=Side(border_style='medium', color='FF000000'))
            bottom_thick_border = Border(bottom=Side(border_style='medium', color='FF000000'))
            right_thick_border = Border(right=Side(border_style='medium', color='FF000000'))

            heading_border = Border(left=Side(border_style='medium', color='FF000000'),
                                    right=Side(border_style='medium', color='FF000000'),
                                    top=Side(border_style='medium', color='FF000000'),
                                    bottom=Side(border_style='medium', color='FF000000'))
            
            bottom_right_thick_border = Border(right=Side(border_style='medium', color='FF000000'),
                                                bottom=Side(border_style='medium', color='FF000000'))

            for row in worksheet.iter_rows(min_row=1, max_row=counter-2, min_col=1, max_col=1):
                for cell in row:
                    cell.border = left_thick_border

            for row in worksheet.iter_rows(min_row=1, max_row=counter-2, min_col=10, max_col=10):
                for cell in row:
                    cell.border = right_thick_border

            for row in worksheet.iter_rows(min_row=counter-2, max_row=counter-2, min_col=1, max_col=10):
                for cell in row:
                    cell.border = bottom_thick_border
            
            for row in worksheet.iter_rows(min_row=counter, max_row=counter+5, min_col=1, max_col=10):
                for cell in row:
                    cell.border = border

            for row in worksheet.iter_rows(min_row=1, max_row=1, min_col=1, max_col=10):
                for cell in row:
                    cell.border = heading_border

            #assign bottom right thick border to J19
            worksheet['J{}'.format(str(counter-2))].border = bottom_right_thick_border

            # Auto-adjusting column widths
            for column_cells in worksheet.columns:
                length = max(len(str(cell.value)) for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = length

            # Centering text in every cell
            for row in worksheet.iter_rows():
                for cell in row:
                    cell.alignment = Alignment(horizontal='center', vertical='center')



    summary_df = pd.DataFrame(summary_dict_list)

    summary_df['ZNE Hellinger fidelity % change'] = (summary_df['ZNE Hellinger fidelity'] - summary_df['Initial Hellinger fidelity']) / summary_df['Initial Hellinger fidelity'] * 100
    summary_df['ZNE TV % change'] = (summary_df['Initial TV'] - summary_df['ZNE TV']) / summary_df['Initial TV'] * 100
    summary_df['FFZNE Hellinger fidelity % change'] = (summary_df['FFZNE Hellinger fidelity'] - summary_df['Initial Hellinger fidelity']) / summary_df['Initial Hellinger fidelity'] * 100
    summary_df['FFZNE TV % change'] = (summary_df['Initial TV'] - summary_df['FFZNE TV']) / summary_df['Initial TV'] * 100
    summary_df['RBZNE Hellinger fidelity % change'] = (summary_df['RBZNE Hellinger fidelity'] - summary_df['Initial Hellinger fidelity']) / summary_df['Initial Hellinger fidelity'] * 100
    summary_df['RBZNE TV % change'] = (summary_df['Initial TV'] - summary_df['RBZNE TV']) / summary_df['Initial TV'] * 100

    summary_df = summary_df[['N','T','Initial Hellinger fidelity','Initial TV',
                            'ZNE Hellinger fidelity','ZNE Hellinger fidelity % change','ZNE TV','ZNE TV % change',
                            'FFZNE Hellinger fidelity','FFZNE Hellinger fidelity % change','FFZNE TV','FFZNE TV % change',
                            'RBZNE Hellinger fidelity','RBZNE Hellinger fidelity % change','RBZNE TV','RBZNE TV % change']]

    sheet_name = 'Summary'

    with pd.ExcelWriter("../results/{}_distribution_data.xlsx".format(system),mode='a', engine='openpyxl') as writer:
            summary_df.to_excel(writer, sheet_name='Summary', index=False)
            workbook = writer.book
            worksheet = writer.sheets[sheet_name]
            
            # Accessing the active sheet
            active_sheet = workbook.active

            # Calculate Geometric mean of 1 + (% change / 100) for Hellinger fidelity and TV
            worksheet['A24'] = 'Geometric mean'
            worksheet['F24'] = (1 + summary_df['ZNE Hellinger fidelity % change'] / 100).prod() ** (1 / len(summary_df))
            worksheet['H24'] = (1 + summary_df['ZNE TV % change'] / 100).prod() ** (1 / len(summary_df))
            worksheet['J24'] = (1 + summary_df['FFZNE Hellinger fidelity % change'] / 100).prod() ** (1 / len(summary_df))
            worksheet['L24'] = (1 + summary_df['FFZNE TV % change'] / 100).prod() ** (1 / len(summary_df))
            worksheet['N24'] = (1 + summary_df['RBZNE Hellinger fidelity % change'] / 100).prod() ** (1 / len(summary_df))
            worksheet['P24'] = (1 + summary_df['RBZNE TV % change'] / 100).prod() ** (1 / len(summary_df))

            # Auto-adjusting column widths
            for column_cells in worksheet.columns:
                length = max(len(str(cell.value)) for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = length

            # Centering text in every cell
            for row in worksheet.iter_rows():
                for cell in row:
                    cell.alignment = Alignment(horizontal='center', vertical='center')

Processing system: ibm_kyoto
Number of jobs: 14
Processing N: 4, T: 1


NameError: name 'result' is not defined

### Do ZNE and RBZNE where it matters the most

In [138]:
distribution_data_df['Percentage Difference in Simulation and Ideal'] = (distribution_data_df['Simulation probability'] - distribution_data_df['Ideal probability'])/distribution_data_df['Ideal probability'] * 100
distribution_data_df[['State','Simulation probability','Ideal probability','Percentage Difference in Simulation and Ideal']]

,State,Simulation probability,Ideal probability,Percentage Difference in Simulation and Ideal
15,1111,0.197998,0.117405,68.644960
14,1110,0.124268,0.039047,218.253080
0,0000,0.105713,0.210729,-49.834776
3,0011,0.101318,0.099972,1.346508
12,1100,0.078857,0.099972,-21.120670
8,1000,0.061768,0.109919,-43.806518
7,0111,0.060791,0.039047,55.687656
9,1001,0.053955,0.080880,-33.289800
2,0010,0.041260,0.005167,698.488562
13,1101,0.028809,0.034286,-15.974602
